# Graph RAG using LangChain and Kuzu

This notebook demonstrates **Graph-based Retrieval Augmented Generation** using:
- KuzuDB as the graph database
- LangChain for graph construction and querying
- LLMs for knowledge graph extraction

Key features:
- Converts text into structured knowledge graphs
- Supports complex relationship queries
- Visualizes the resulting graph

Based on: [langchain-kuzu package](https://pypi.org/project/langchain-kuzu/)

In [ ]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f

# cSpell: disable

## Import Required Libraries

Key components we'll use:
- `kuzu`: Graph database engine
- `LLMGraphTransformer`: Converts text to graph structure
- `KuzuGraph`: LangChain wrapper for KuzuDB
- `KuzuQAChain`: Handles graph-based question answering
- `CytoscapeWidget`: For graph visualization

In [ ]:
import kuzu
from genai_tk.core.llm_factory import get_llm
from ipycytoscape import CytoscapeWidget
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_kuzu.chains.graph_qa.kuzu import KuzuQAChain
from langchain_kuzu.graphs.kuzu_graph import KuzuGraph
from langchain_openai import ChatOpenAI

## Build the Knowledge Graph

We'll create a graph from text by:
1. Defining allowed node types (Person, Company, Location)
2. Specifying allowed relationships between them
3. Using an LLM to extract entities and relationships
4. Storing the structured graph in KuzuDB

The example text contains facts about Apple's CEO and headquarters location.

In [ ]:
DB = "/tmp/test_db4"
MODEL_ID = None
llm = get_llm(llm=MODEL_ID)

## Query the Knowledge Graph

Now we'll use the `KuzuQAChain` to:
1. Accept natural language questions
2. Generate and execute Cypher queries
3. Return formatted answers

The chain shows the generated Cypher queries when verbose=True.

In [ ]:
llm = get_llm(llm=None)
db = kuzu.Database(DB)
graph = KuzuGraph(db, allow_dangerous_requests=True)

## Visualize the Graph

We'll use Cytoscape to render an interactive visualization of:
- Nodes (entities)
- Edges (relationships)
- Their properties

In [ ]:
# Create Cytoscape widget
from genai_blueprint.webapp.ui_components.cypher_graph_display import get_cytoscape_json, get_cytoscape_style

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_json(get_cytoscape_json(graph))
cyto.set_style(get_cytoscape_style())
# Set layout and style
cyto.set_layout(animate=True)

In [ ]:
# Display the graph
cyto